In [1]:
# use BioSent2Vec and traditional ml on titles

import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance
import numpy as np
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


ValueError: numpy.ufunc has the wrong size, try recompiling. Expected 192, got 216

In [4]:
neg_path = 'titledata/rt-polarity.neg-ns'
pos_path = 'titledata/rt-polarity.pos-ns'
model_path = 'BioSentVec_PubMed_MIMICIII-bigram_d700.bin'
model = sent2vec.Sent2vecModel()

In [5]:
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

model successfully loaded


In [6]:
# prepare the data from one single file
# split into 0.9/0.1, first titles
pos_arrays = np.zeros((13018, 700)) # the sum of all the training examples (neg+pos)
pos_labels = np.zeros(13018)

neg_arrays = np.zeros((13018, 700)) # the sum of all the training examples (neg+pos)
neg_labels = np.zeros(13018)

In [7]:
with open(pos_path) as pos:
    i = 0
    for line in pos:
    	sentence_vector = model.embed_sentence(line)
    	pos_arrays[i] = sentence_vector
    	pos_labels[i] = 1
    	i = i + 1
    print(i)

13018


In [8]:
with open(neg_path) as neg:
    i = 0
    for line in neg:
    	sentence_vector = model.embed_sentence(line)
    	neg_arrays[i] = sentence_vector
    	neg_labels[i] = 0
    	i = i + 1
    print(i)

13018


In [10]:
# https://stackoverflow.com/questions/3674409/how-to-split-partition-a-dataset-into-training-and-test-datasets-for-e-g-cros
np.random.shuffle(pos_arrays)
training_pos, test_pos = pos_arrays[:11716,:], pos_arrays[11716:,:]

np.random.shuffle(neg_arrays)
training_neg, test_neg = neg_arrays[:11716,:], neg_arrays[11716:,:]

train_arrays = np.concatenate([training_pos, training_neg])
print(np.shape(train_arrays))

(23432, 700)


In [11]:
train_pos_label = np.ones(11716)
train_neg_label = np.zeros(11716)
train_labels = np.concatenate([train_pos_label, train_neg_label])

test_arrays = np.concatenate([test_pos, test_neg])

test_pos_label = np.ones(13018 - 11716)
test_neg_label = np.zeros(13018 - 11716)
test_labels = np.concatenate([test_pos_label, test_neg_label])

In [12]:
print(len(train_arrays))

23432


In [15]:
# SVM
from sklearn import svm
X = train_arrays # e.g. X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = train_labels # y = np.array([1, 1, 2, 2])
C_2d_range = [0.0005, 0.01, 0.05, 0.1, 0.2, 0.5, 1,2,4,8,16]
gamma_2d_range = [0.0001, 0.001, 0.01, 0.1, 1,10, 100, 1000]

for C in C_2d_range:
    for gamma in gamma_2d_range:
        print(C)
        print(gamma)
        clf = svm.SVC(C=C, gamma=gamma)
        clf.fit(X, y)
        predition = clf.predict(test_arrays)
        print(clf.score(test_arrays, test_labels))
        p = precision_score(test_labels, predition)
        r = recall_score(test_labels, predition)
        print(p)
        print(r)
        f = 2 * p * r / (p + r)
        print(f)
        print(clf.score(train_arrays, train_labels))

0.0005
0.0001
0.6344086021505376
0.8289473684210527
0.3387096774193548
0.48091603053435117
0.6371628542164561
0.0005
0.001
0.6501536098310292
0.8199672667757774
0.3847926267281106
0.5237846314688971
0.6504353021509047
0.0005
0.01
0.7092933947772657
0.746606334841629
0.6336405529953917
0.6855006231823847
0.7138955274837828
0.0005
0.1
0.586405529953917
0.5495376486129459
0.9585253456221198
0.6985726280436608
0.582579378627518
0.0005
1
0.500768049155146
0.5003843197540354
1.0
0.6670081967213115
0.5011949470809149
0.0005
10
0.500768049155146
0.5003843197540354
1.0
0.6670081967213115
0.9999573233185387
0.0005
100
0.500768049155146
0.5003843197540354
1.0
0.6670081967213115
0.9999573233185387
0.0005
1000
0.500768049155146
0.5003843197540354
1.0
0.6670081967213115
0.9999573233185387
0.01
0.0001
0.6344086021505376
0.8289473684210527
0.3387096774193548
0.48091603053435117
0.6371628542164561
0.01
0.001
0.6501536098310292
0.8199672667757774
0.3847926267281106
0.5237846314688971
0.6504353021509047


0.9999573233185387
16
100
0.500768049155146
0.5003843197540354
1.0
0.6670081967213115
0.9999573233185387
16
1000
0.500768049155146
0.5003843197540354
1.0
0.6670081967213115
0.9999573233185387


In [29]:
# random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

X = train_arrays
y = train_labels

depth = list(range(2, 30, 2))
num_of_trees = list(range(50, 800, 50)) 

for d in depth:
    for n in num_of_trees:
        print(d)
        print(n)
        clf = RandomForestClassifier(n_estimators=n, max_depth=d, random_state=0)
        clf.fit(X, y)
        predition = clf.predict(test_arrays)
        print(clf.score(test_arrays, test_labels))
        p = precision_score(test_labels, predition)
        r = recall_score(test_labels, predition)
        print(p)
        print(r)
        f = 2 * p * r / (p + r)
        print(f)
        print(clf.score(train_arrays, train_labels))
        

2
50
0.6981566820276498
0.6954545454545454
0.7050691244239631
0.700228832951945
0.7035250938886992
2
100
0.705837173579109
0.7033383915022762
0.7119815668202765
0.7076335877862595
0.7092437692045066
2
150
0.7092933947772657
0.7050413844996237
0.7196620583717358
0.7122767008741921
0.7112495732331854
2
200
0.706989247311828
0.7043214556482184
0.7135176651305684
0.7088897367417016
0.7094998292932742
2
250
0.7035330261136713
0.7032208588957055
0.7043010752688172
0.7037605525709901
0.7092437692045066
2
300
0.7089093701996928
0.7066869300911854
0.7142857142857143
0.7104660045836516
0.7089877091157392
2
350
0.7073732718894009
0.7051671732522796
0.7127496159754224
0.7089381207028266
0.7097132127005804
2
400
0.7085253456221198
0.7055261165783497
0.7158218125960062
0.7106366755623332
0.7097985660635029
2
450
0.7123655913978495
0.7077385424492862
0.7235023041474654
0.7155336118496012
0.7094571526118129
2
500
0.7123655913978495
0.708994708994709
0.7204301075268817
0.7146666666666667
0.710566746329

0.9746500512120178
12
400
0.7442396313364056
0.7317784256559767
0.7711213517665131
0.7509349289454003
0.9748207579378627
12
450
0.7434715821812596
0.7313868613138687
0.7695852534562212
0.7500000000000001
0.9749061113007853
12
500
0.7423195084485407
0.7284576393917451
0.7726574500768049
0.7499068207230712
0.9749061113007853
12
550
0.7423195084485407
0.729121278140886
0.7711213517665131
0.7495334079880551
0.9748207579378627
12
600
0.7442396313364056
0.7307692307692307
0.7734254992319508
0.7514925373134328
0.9746500512120178
12
650
0.7465437788018433
0.7336244541484717
0.7741935483870968
0.7533632286995516
0.975034141345169
12
700
0.7461597542242704
0.7337709700948213
0.7726574500768049
0.752712308267864
0.974863434619324
12
750
0.7450076804915514
0.7321688500727802
0.7726574500768049
0.7518684603886396
0.974692727893479
14
50
0.7323348694316436
0.7255779269202088
0.7473118279569892
0.7362845251608022
0.9885626493683851
14
100
0.7323348694316436
0.7222630418809699
0.7549923195084486
0.738

0.99965858654831
22
700
0.7327188940092166
0.7173601147776184
0.7680491551459293
0.7418397626112759
0.99965858654831
22
750
0.7354070660522274
0.7203450754852624
0.7695852534562212
0.7441515038989974
0.99965858654831
24
50
0.7261904761904762
0.7148067104303428
0.7526881720430108
0.7332585110362888
0.9996159098668488
24
100
0.7327188940092166
0.7151988636363636
0.7734254992319508
0.7431734317343174
0.9999146466370775
24
150
0.7407834101382489
0.7240886347390993
0.7780337941628265
0.7500925583117363
0.9999146466370775
24
200
0.7365591397849462
0.7206303724928367
0.7726574500768049
0.7457375833951074
0.9999146466370775
24
250
0.7396313364055299
0.7225392296718973
0.7780337941628265
0.7492603550295859
0.9999146466370775
24
300
0.7403993855606759
0.721045197740113
0.7841781874039938
0.7512877115526121
0.9999146466370775
24
350
0.7403993855606759
0.7207334273624824
0.7849462365591398
0.7514705882352941
0.9999146466370775
24
400
0.7423195084485407
0.7220267417311752
0.7880184331797235
0.75358

In [16]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

X = train_arrays
y = train_labels

for k in list(range(5, 100, 5)):
    print(k)
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X, y) 
    predition = neigh.predict(test_arrays)
    print(neigh.score(test_arrays, test_labels))
    p = precision_score(test_labels, predition)
    r = recall_score(test_labels, predition)
    print(p)
    print(r)
    f = 2 * p * r / (p + r)
    print(f)
    print(neigh.score(train_arrays, train_labels))

5
0.7334869431643625
0.7512396694214876
0.6981566820276498
0.7237261146496815
0.8118385114373506
10
0.7131336405529954
0.7610536218250236
0.6213517665130568
0.6841437632135308
0.7690764766131786
15
0.7265745007680492
0.7478991596638656
0.6835637480798771
0.7142857142857142
0.7682229429839535
20
0.7242703533026114
0.7669104204753199
0.6443932411674347
0.7003338898163606
0.7549931717309663
25
0.7296466973886329
0.7559931506849316
0.6781874039938556
0.7149797570850203
0.7567429156708775
30
0.7238863287250384
0.7614349775784753
0.652073732718894
0.7025237898220935
0.7476954592010925
35
0.7208141321044547
0.7497827975673328
0.662826420890937
0.7036282103546677
0.7487623762376238
40
0.7188940092165899
0.7572202166064982
0.6443932411674347
0.6962655601659752
0.7416780471150564
45
0.7185099846390169
0.748471615720524
0.6582181259600615
0.7004495300367797
0.7449641515875726
50
0.7162058371735791
0.757548032936871
0.6359447004608295
0.6914405010438414
0.7399283031751451
55
0.7185099846390169
0.7

In [17]:
from sklearn.linear_model import LogisticRegression
#logistic regression
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)
predition = classifier.predict(test_arrays)
print(classifier.score(test_arrays, test_labels))
p = precision_score(test_labels, predition)
r = recall_score(test_labels, predition)
print(p)
print(r)
f = 2 * p * r / (p + r)
print(f)
print(classifier.score(train_arrays, train_labels))

/home/paperspace/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7377112135176651
0.7318352059925094
0.7503840245775729
0.7409935532802427
0.7593888699214749
